In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import glob
import matplotlib.pylab as plt

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id

from general_tools.notebook.tf import reset_tf_graph                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k, indices_in_iterable

from geo_tool import Point_Cloud

from tf_lab.nips.shape_net import pc_loader as sn_pc_loader

from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs

from tf_lab.nips.evaluate_gan import entropy_of_occupancy_grid, point_cloud_distances,\
                                     jensen_shannon_divergence, sample_pclouds_distances

from general_tools.strings import trim_content_after_last_dot

from general_tools.plotting import stack_images_in_square_grid

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_samples = 2048
seed = 42
voxel_resolution = 32
np.random.seed(seed)
save_figs = False
cmp_with_noise = False

In [5]:
# Load Ground-Truth Data
in_f = '/orions4-zfs/projects/lins2/Panos_Space/DATA/NIPS/our_samples/gt_all_chair.npz'
gt_data = np.load(in_f)
gt_data = gt_data[gt_data.keys()[0]]
_, gt_grid_var = entropy_of_occupancy_grid(gt_data, voxel_resolution)

In [6]:
# TESTING RANDOM NOISE
# Chairs 32 voxel-resolution: Gauss = 0.36, Uni=0.38
if cmp_with_noise:
    import scipy.stats as stats
    lower, upper = -0.5, 0.5
    mu, sigma = 0, 1
    # X = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
    # size = (len(gt_data), n_pc_samples, 3)
    # random_pcs = X.rvs(np.prod(size))
    # random_pcs = random_pcs.reshape(size)
    random_pcs = np.random.uniform(low=-0.5, high=0.5, size=(len(gt_data), n_pc_samples, 3))
    ## for i, pc in enumerate(random_pcs):
    ##     random_pcs[i] = Point_Cloud(pc).center_axis()[0].points
    
    _, bline_grid_var = entropy_of_occupancy_grid(random_pcs, voxel_resolution)
    print jensen_shannon_divergence(bline_grid_var, gt_grid_var)

In [40]:
# LOAD (Synthetic) Point-Cloud Data
# exp_name = 'ae_trained_200eps_512gen_ae_emd_1c_gan_chair_2048_pts_epoch_9'
exp_name = 'v0_raw_gan_chair_2048_pts_epoch_30'

in_f = '/orions4-zfs/projects/lins2/Panos_Space/DATA/NIPS/our_samples/'
in_f = osp.join(in_f, exp_name + '.npz')
# in_f = '/orions4-zfs/projects/lins2/Panos_Space/DATA/NIPS/mit_3dgan_1K_synthetic_samples/point_clouds/2048/chair.npz'
top_fig_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/NIPS/Images/'

syn_data = np.load(in_f)
syn_data = syn_data[syn_data.keys()[0]]

look_only = len(syn_data)
# look_only = 1000 # speed up
syn_data = syn_data[:look_only]

print syn_data.shape
print np.min(syn_data), np.max(syn_data)

## One way to push syn_data in u-cube
cleaner =[]
rem = 0
for i in syn_data:
    if max(abs(np.max(i)), abs(np.min(i))) <= 0.5:
#         cleaner.append(Point_Cloud(i).center_axis()[0].points)
        cleaner.append(i)
    else:
        rem += 1
print 'removed', rem
syn_data = np.array(cleaner)

(6778, 2048, 3)
-0.418762 0.40793
removed 0


In [37]:
pclouds = syn_data.copy()

pclouds = pclouds - np.expand_dims(np.mean(pclouds, axis=1), 1)
dist = np.max(np.sqrt(np.sum(pclouds ** 2, axis=2)), 1)
dist = np.expand_dims(np.expand_dims(dist, 1), 2)
pclouds = pclouds / dist

mentropy, sd_grid_var = entropy_of_occupancy_grid(pclouds, voxel_resolution)
print mentropy
print jensen_shannon_divergence(gt_grid_var, sd_grid_var)

In [46]:
pclouds = syn_data.copy()

pclouds = pclouds - np.expand_dims(np.mean(pclouds, axis=1), 1)
dist = np.max(np.sqrt(np.sum(pclouds ** 2, axis=2)), 1)
dist = np.expand_dims(np.expand_dims(dist, 1), 2)
pclouds = pclouds / dist

all_dists = sample_pclouds_distances(pclouds, 30, 100, 'emd')
print np.median(all_dists), np.mean(all_dists)

4327.09487915 4346.75127869


# Pairwise Distances

In [26]:
# check_data = pclouds
# distance_type = 'chamfer'
# all_dists = point_cloud_distances(check_data, 10, distance_type)
print np.median(all_dists)

# plt.hist(all_dists)[2]
# plt.xlabel(distance_type + ' pairwise distances')
# plt.ylabel('Histogram')
# if save_figs:
#     plt.savefig(trim_content_after_last_dot(in_f) + '_dist_hist.png')

0.0480847


# Plotting data in grid

In [ ]:
create_dir('images_' + exp_name)
size = 100
rids = np.random.choice(range(len(syn_data)), size=size, replace=False)
plt.ioff()

for i in range(size):
    fig = Point_Cloud(syn_data[rids[i]]).plot(show=False, show_axis=False, azim=290, in_u_sphere=True, s=20);
    fig.savefig(osp.join('images_' + exp_name, 'sample_' +str(i)))
    plt.close()    

file_names = glob.glob('images_' + exp_name +'/*.png')
stack_images_in_square_grid(file_names, save_file= exp_name + '_BIG.png')